<a href="https://colab.research.google.com/github/humazafar2703/Assignment3_Python/blob/main/Copy_of_Rainfall_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [30]:
# Step 1: Install required packages
!pip install kagglehub geopy --no-cache-dir

# Step 2: Import libraries
import kagglehub
import os
import zipfile
import pandas as pd
from geopy.geocoders import Nominatim
import time


In [31]:

# Download the dataset and get the path
uk_rainfall = kagglehub.dataset_download("jakewright/2m-daily-weather-history-uk")
print("Dataset downloaded to:", uk_rainfall)

# Check files in the dataset folder
print("Files in dataset folder:", os.listdir(uk_rainfall))

Dataset downloaded to: /kaggle/input/2m-daily-weather-history-uk
Files in dataset folder: ['all_weather_data.csv', 'all_weather_data.parquet']


In [32]:
df['date'] = pd.to_datetime(df['date'], errors='coerce')
df = df.dropna(subset=['date'])

start_date = df['date'].min()
end_date = df['date'].max()

print(f"📅 Data covers from {start_date.date()} to {end_date.date()}")


📅 Data covers from 2009-01-01 to 2024-11-12


In [33]:
import zipfile
import pandas as pd

# Assuming uk_rainfall is your dataset folder path
print("Files in download path:")
print(os.listdir(uk_rainfall))

# Find all ZIP files (case-insensitive)
zip_files = [f for f in os.listdir(uk_rainfall) if f.lower().endswith('.zip')]
print("Zip files found:", zip_files)

# Extract all ZIP files found
for zip_file in zip_files:
    zip_path = os.path.join(uk_rainfall, zip_file)
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(uk_rainfall)
    print(f"Extracted {zip_file}")

# List files after extraction
print("Files after extraction:")
print(os.listdir(uk_rainfall))

# Find all CSV files (case-insensitive)
csv_files = [f for f in os.listdir(uk_rainfall) if f.lower().endswith('.csv')]
print("CSV files found:", csv_files)

# Load the first CSV file if any found
if csv_files:
    csv_path = os.path.join(uk_rainfall, csv_files[0])
    df = pd.read_csv(csv_path)
    print(f"Loaded '{csv_files[0]}':")
    print(df.head())
else:
    print("No CSV files found in the dataset folder!")


Files in download path:
['all_weather_data.csv', 'all_weather_data.parquet']
Zip files found: []
Files after extraction:
['all_weather_data.csv', 'all_weather_data.parquet']
CSV files found: ['all_weather_data.csv']
Loaded 'all_weather_data.csv':
      location        date  min_temp °c  max_temp °c  rain mm  humidity %  \
0     Holywood  2009-01-01          0.0          3.0      0.0        86.0   
1   North Cray  2009-01-01         -3.0          2.0      0.0        93.0   
2  Portknockie  2009-01-01          2.0          4.0      0.8        88.0   
3  Blairskaith  2009-01-01         -3.0          1.0      0.0        86.0   
4     Onehouse  2009-01-01         -1.0          3.0      0.0        91.0   

   cloud_cover %  wind_speed km/h wind_direction  wind_direction_numerical  
0           14.0             12.0              E                      90.0  
1           44.0              8.0            NNE                      22.5  
2           87.0             10.0            ESE           

In [34]:
 #Now check unique locations
num_unique_locations = len(df['location'].unique())
print(f"Number of unique locations: {num_unique_locations}")

Number of unique locations: 504


In [35]:
!pip install geopy


In [36]:
from geopy.geocoders import Nominatim
import time
from geopy.geocoders import Nominatim
import time

geolocator = Nominatim(user_agent="uk-weather-locator", timeout=10)

def get_lat_lon(place):
    try:
        location = geolocator.geocode(place + ", UK")
        if location:
            return location.latitude, location.longitude
    except:
        return None, None
    return None, None

# Step 10: Generate location → (lat, lon) dictionary
locations = df['location'].unique()
location_coords = {}

print("\nFetching coordinates for each location:\n")
for place in locations:
    lat, lon = get_lat_lon(place)
    location_coords[place] = (lat, lon)
    print(f"{place}: {lat}, {lon}")
    time.sleep(1)  # To respect OpenStreetMap rate limit

# Step 11: Preview result
print("\nSample of location coordinates:")
for loc in list(location_coords.items())[:10]:
    print(loc)


Fetching coordinates for each location:

Holywood: 54.6404494, -5.8361723
North Cray: 51.4251795, 0.1345992
Portknockie: 57.7034964, -2.8581483
Blairskaith: 55.952868, -4.2515364
Onehouse: 52.1975136, 0.9543305
Poling Corner: 50.8422347, -0.5138469
Rhydyfelin: 51.5849564, -3.3064646
Old Felixstowe: 51.9702624, 1.3671791
Vauxhall: 51.4874834, -0.1229297
Pencraig: 52.2247312, -3.1002924
Carrickfergus: 54.7136165, -5.8073112
Parkway: 53.4473192, -2.2454801
Mutford: 52.4359451, 1.6513565
Eden: 54.6056052, -2.6715019
Madrid: 40.4608258, -3.4719145
Tyntetown: None, None
Pillerton Priors: 52.1265993, -1.5735999
Ballymacarret: 54.5978097, -5.8974846
Pulborough: 50.9575636, -0.5097654
Dufftown: 57.4449043, -3.1283484
Reigate: 51.2376557, -0.2057067
Pont Rhyd-Y-Cyff: 51.5904461, -3.623818
Paddington: 51.5163047, -0.1792423
Oakley: 52.1710689, -0.524367
Palgrave: 52.3644842, 1.1044938
Oslo: 51.5471622, -0.0556001
Lisson Grove: 51.5266514, -0.1723981
Newton: 52.0322303, 0.7943311
Nayland: 51.9721

In [37]:
# Assuming location_coords = {location: (lat, lon), ...}

# Create a DataFrame for locations and their coordinates
loc_coords_df = pd.DataFrame([
    {'location': loc, 'latitude': coords[0], 'longitude': coords[1]}
    for loc, coords in location_coords.items()
])

# Filter locations inside UK bounding box
uk_loc_df = loc_coords_df[
    (loc_coords_df['latitude'] >= 49.5) & (loc_coords_df['latitude'] <= 61.0) &
    (loc_coords_df['longitude'] >= -8.5) & (loc_coords_df['longitude'] <= 2.0)
]

uk_locations = uk_loc_df['location'].tolist()

print(f"Number of locations within UK bounds: {len(uk_locations)}")


Number of locations within UK bounds: 477


In [38]:
# Aggregate rainfall stats by location
rainfall_stats = df.groupby('location')['rain mm'].agg(['min', 'mean', 'max']).reset_index()
rainfall_stats.rename(columns={'min':'min_rain_mm', 'mean':'avg_rain_mm', 'max':'max_rain_mm'}, inplace=True)


In [39]:
rainfall_stats['latitude'] = rainfall_stats['location'].apply(lambda loc: location_coords.get(loc, (None, None))[0])
rainfall_stats['longitude'] = rainfall_stats['location'].apply(lambda loc: location_coords.get(loc, (None, None))[1])

# Drop rows without coordinates
rainfall_stats = rainfall_stats.dropna(subset=['latitude', 'longitude'])


In [40]:
import plotly.express as px

# Assuming you have rainfall_stats and location_coords from previous steps

# Add latitude and longitude from location_coords
rainfall_stats['latitude'] = rainfall_stats['location'].apply(lambda loc: location_coords.get(loc, (None, None))[0])
rainfall_stats['longitude'] = rainfall_stats['location'].apply(lambda loc: location_coords.get(loc, (None, None))[1])

# Drop rows with missing coordinates
rainfall_stats = rainfall_stats.dropna(subset=['latitude', 'longitude'])

# Create scatter map with average rainfall (color + size)
fig_avg = px.scatter_mapbox(
    rainfall_stats,
    lat='latitude',
    lon='longitude',
    hover_name='location',
    hover_data={
        'min_rain_mm': ':.2f',
        'avg_rain_mm': ':.2f',
        'max_rain_mm': ':.2f'
    },
    color='avg_rain_mm',
    size='avg_rain_mm',
    color_continuous_scale='Blues',
    size_max=20,
    zoom=5,
    height=600,
    title="Spatial Distribution of Average Rainfall (mm) in UK"
)
fig_avg.update_layout(mapbox_style='carto-positron', margin={"r":0,"t":50,"l":0,"b":0})
fig_avg.show()

# similar plots for min and max rainfall

fig_min = px.scatter_mapbox(
    rainfall_stats,
    lat='latitude',
    lon='longitude',
    hover_name='location',
    hover_data={'min_rain_mm': ':.2f'},
    color='min_rain_mm',
    size='min_rain_mm',
    color_continuous_scale='Greens',
    size_max=20,
    zoom=5,
    height=600,
    title="Spatial Distribution of Minimum Rainfall (mm) in UK"
)
fig_min.update_layout(mapbox_style='carto-positron', margin={"r":0,"t":50,"l":0,"b":0})
fig_min.show()

fig_max = px.scatter_mapbox(
    rainfall_stats,
    lat='latitude',
    lon='longitude',
    hover_name='location',
    hover_data={'max_rain_mm': ':.2f'},
    color='max_rain_mm',
    size='max_rain_mm',
    color_continuous_scale='Reds',
    size_max=20,
    zoom=5,
    height=600,
    title="Spatial Distribution of Maximum Rainfall (mm) in UK"
)
fig_max.update_layout(mapbox_style='carto-positron', margin={"r":0,"t":50,"l":0,"b":0})
fig_max.show()


In [41]:
# Step 1: Extract year from date
df['year'] = df['date'].dt.year

# Step 2: Sum rainfall by location and year
yearly_rainfall = df.groupby(['location', 'year'])['rain mm'].sum().reset_index()

# Step 3: Find year with max rainfall per location
max_rain_year = yearly_rainfall.loc[yearly_rainfall.groupby('location')['rain mm'].idxmax()]
max_rain_year.rename(columns={'year': 'year_max_rain', 'rain mm': 'max_rainfall_mm'}, inplace=True)

# Step 4: Find year with min rainfall per location
min_rain_year = yearly_rainfall.loc[yearly_rainfall.groupby('location')['rain mm'].idxmin()]
min_rain_year.rename(columns={'year': 'year_min_rain', 'rain mm': 'min_rainfall_mm'}, inplace=True)

# Step 5: Merge max and min tables
rainfall_years = pd.merge(max_rain_year[['location', 'year_max_rain', 'max_rainfall_mm']],
                          min_rain_year[['location', 'year_min_rain', 'min_rainfall_mm']],
                          on='location')

# Display a sample
print(rainfall_years.head())


AttributeError: Can only use .dt accessor with datetimelike values

In [ ]:
avg_rainfall = df.groupby('location')['rain mm'].mean().reset_index()
avg_rainfall.rename(columns={'rain mm': 'avg_rain_mm'}, inplace=True)


In [ ]:
top_wettest = avg_rainfall.sort_values(by='avg_rain_mm', ascending=False).head(10)
print("Top 10 Wettest Places (by average rainfall):")
print(top_wettest)


In [ ]:
driest = avg_rainfall.sort_values(by='avg_rain_mm', ascending=True).head(10)
print("Top 10 Driest Places (by average rainfall):")
print(driest)


In [ ]:
import plotly.graph_objects as go

def show_table(df, title):
    fig = go.Figure(data=[go.Table(
        header=dict(values=list(df.columns),
                    fill_color='lightblue',
                    align='left'),
        cells=dict(values=[df[col] for col in df.columns],
                   fill_color='white',
                   align='left'))
    ])
    fig.update_layout(title=title)
    fig.show()

show_table(top_wettest, "Top 10 Wettest Places in UK")
show_table(driest, "Top 10 Driest Places in UK")


In [ ]:
# MODEL PHASES

# Ensure 'date' is datetime type
df['date'] = pd.to_datetime(df['date'])

# Set date as index temporarily for resampling
df.set_index('date', inplace=True)

# Aggregate weekly rainfall sums per location
df_weekly = df.groupby('location').resample('W')['rain mm'].sum().reset_index()

print(df_weekly.head())


In [ ]:
#  2. Time-based Split into Train / Validation / Test
# We split based on weeks (after aggregation), preserving time order to avoid data leakage.

# Sort by date
df_weekly = df_weekly.sort_values('date')

# Get unique sorted weeks
weeks = df_weekly['date'].sort_values().unique()

# Define split indices
train_end = int(len(weeks) * 0.7)
val_end = int(len(weeks) * 0.85)

train_weeks = weeks[:train_end]
val_weeks = weeks[train_end:val_end]
test_weeks = weeks[val_end:]

# Split dataframes by weeks
train_df = df_weekly[df_weekly['date'].isin(train_weeks)]
val_df = df_weekly[df_weekly['date'].isin(val_weeks)]
test_df = df_weekly[df_weekly['date'].isin(test_weeks)]

print(f"Train weeks: {len(train_weeks)}")
print(f"Validation weeks: {len(val_weeks)}")
print(f"Test weeks: {len(test_weeks)}")

print("Train sample:")
print(train_df.head())



In [ ]:
## had to re import because it would crash and not carry forward from previous import

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import StandardScaler
from torch_geometric.nn import GCNConv
from torch_geometric.data import Data as GeoData
import tensorflow as tf
from tensorflow.keras import layers


In [ ]:
#Step 1: Prepare sequences and datasets
SEQ_LEN = 4
PRED_LEN = 1

def create_sequences(df):
    """
    Create (X,y) for each location: X=last SEQ_LEN rain, y=next PRED_LEN rain
    Returns list of dicts with keys: location, X, y, date
    """
    sequences = []
    for loc in df['location'].unique():
        loc_data = df[df['location'] == loc].sort_values('date')
        rain = loc_data['rain mm'].values
        dates = loc_data['date'].values
        for i in range(len(rain) - SEQ_LEN - PRED_LEN + 1):
            X = rain[i:i+SEQ_LEN]
            y = rain[i+SEQ_LEN:i+SEQ_LEN+PRED_LEN]
            sequences.append({'location': loc, 'X': X, 'y': y, 'date': dates[i+SEQ_LEN+PRED_LEN-1]})
    return sequences

train_seq = create_sequences(train_df)
val_seq = create_sequences(val_df)
test_seq = create_sequences(test_df)

print(f"Train sequences: {len(train_seq)}, Val sequences: {len(val_seq)}, Test sequences: {len(test_seq)}")



In [ ]:
#Step 2: Dataset & DataLoader for PyTorch models (GraphCast, RainFormer)

class RainDataset(Dataset):
    def __init__(self, sequences, scaler=None):
        self.sequences = sequences
        self.scaler = scaler
        # Fit scaler on train only if not provided
        if scaler is None:
            all_x = np.vstack([seq['X'] for seq in sequences])
            self.scaler = StandardScaler().fit(all_x)
        # Scale inputs
        for seq in self.sequences:
            seq['X'] = self.scaler.transform([seq['X']])[0]
            seq['y'] = np.array(seq['y'])  # y is small, no scaling here (or scale if desired)

    def __len__(self):
        return len(self.sequences)

    def __getitem__(self, idx):
        seq = self.sequences[idx]
        return torch.tensor(seq['X'], dtype=torch.float32), torch.tensor(seq['y'], dtype=torch.float32)

train_dataset = RainDataset(train_seq)
val_dataset = RainDataset(val_seq, scaler=train_dataset.scaler)
test_dataset = RainDataset(test_seq, scaler=train_dataset.scaler)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32)
test_loader = DataLoader(test_dataset, batch_size=32)



In [ ]:
#Step 3: Model 1 - GenCast (Simple Conditional GAN in TensorFlow)
latent_dim = 8
input_dim = SEQ_LEN
output_dim = PRED_LEN

def build_generator():
    input_seq = layers.Input(shape=(input_dim,))
    noise = layers.Input(shape=(latent_dim,))
    x = layers.Concatenate()([input_seq, noise])
    x = layers.Dense(64, activation='relu')(x)
    x = layers.Dense(output_dim)(x)
    return tf.keras.Model([input_seq, noise], x)

def build_discriminator():
    pred = layers.Input(shape=(output_dim,))
    x = layers.Dense(64, activation='relu')(pred)
    x = layers.Dense(32, activation='relu')(x)
    x = layers.Dense(1, activation='sigmoid')(x)
    return tf.keras.Model(pred, x)

generator = build_generator()
discriminator = build_discriminator()
discriminator.compile(optimizer='adam', loss='binary_crossentropy')

discriminator.trainable = False
input_seq = layers.Input(shape=(input_dim,))
noise = layers.Input(shape=(latent_dim,))
generated = generator([input_seq, noise])
validity = discriminator(generated)
gan = tf.keras.Model([input_seq, noise], validity)
gan.compile(optimizer='adam', loss='binary_crossentropy')

# Training loop (simplified)

import random

def train_gan(epochs=1000, batch_size=32):
    for epoch in range(epochs):
        # Sample real data
        idx = np.random.randint(0, len(train_dataset), batch_size)
        real_X = np.array([train_dataset[i][0].numpy() for i in idx])
        real_y = np.array([train_dataset[i][1].numpy() for i in idx])

        # Generate fake data
        noise = np.random.normal(0, 1, (batch_size, latent_dim))
        fake_y = generator.predict([real_X, noise])

        # Train discriminator
        d_loss_real = discriminator.train_on_batch(real_y, np.ones((batch_size, 1)))
        d_loss_fake = discriminator.train_on_batch(fake_y, np.zeros((batch_size, 1)))
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

        # Train generator
        noise = np.random.normal(0, 1, (batch_size, latent_dim))
        g_loss = gan.train_on_batch([real_X, noise], np.ones((batch_size, 1)))

        if epoch % 100 == 0:
            print(f"Epoch {epoch} - D loss: {d_loss:.4f} - G loss: {g_loss:.4f}")

# Call train_gan() to train



In [ ]:
#Step 4: Model 2 - GraphCast (Graph Neural Network in PyTorch Geometric)
class GraphCastModel(torch.nn.Module):
    def __init__(self, in_channels=SEQ_LEN, hidden_channels=32, out_channels=PRED_LEN):
        super().__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, out_channels)

    def forward(self, x, edge_index):
        x = F.relu(self.conv1(x, edge_index))
        x = self.conv2(x, edge_index)
        return x

# Prepare graph data: node features and edges

# Example: assume node features are sequences of rainfall for each location at current time step (batch of locations)
# edge_index should be constructed based on spatial adjacency of locations (build from location_coords)

# This part requires building edge_index: e.g., connect locations within certain km radius

# Here’s a dummy example edge_index for 3 nodes (replace with your real graph):
edge_index = torch.tensor([[0, 1, 1, 2],
                           [1, 0, 2, 1]], dtype=torch.long)

model = GraphCastModel()

# Example training loop would use batches of node features and edges to predict rainfall



In [ ]:
#Step 6: Evaluation (Example with RMSE)
#Metric	Description
# RMSE	Penalises large errors heavily; good for regression models.
# MAE	Mean absolute error — robust to outliers.
# R² Score	Measures how much variance is explained by the model.
# Correlation	Pearson correlation between true and predicted values.
# Classification Accuracy	Converts rainfall into binary (rain/no-rain) and measures hit rate.


from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np
from scipy.stats import pearsonr

def evaluate_model(model, loader, name='Model'):
    model.eval()
    preds, trues = [], []

    with torch.no_grad():
        for X_batch, y_batch in loader:
            X_batch = X_batch.unsqueeze(-1).permute(1, 0, 2)
            y_true = y_batch.numpy().flatten()

            output = model(X_batch).squeeze().cpu().numpy().flatten()
            preds.extend(output)
            trues.extend(y_true)

    preds = np.array(preds)
    trues = np.array(trues)

    # Metrics
    rmse = mean_squared_error(trues, preds, squared=False)
    mae = mean_absolute_error(trues, preds)
    r2 = r2_score(trues, preds)
    corr, _ = pearsonr(trues, preds)

    # Categorical evaluation: was there rain?
    true_rain = (trues > 0.1).astype(int)
    pred_rain = (preds > 0.1).astype(int)
    binary_accuracy = np.mean(true_rain == pred_rain)

    print(f"\n📊 Evaluation of {name}:")
    print(f"  RMSE: {rmse:.4f}")
    print(f"  MAE:  {mae:.4f}")
    print(f"  R²:   {r2:.4f}")
    print(f"  Pearson Correlation: {corr:.4f}")
    print(f"  Rain/No-Rain Accuracy: {binary_accuracy:.4f}")

    return {
        "rmse": rmse,
        "mae": mae,
        "r2": r2,
        "corr": corr,
        "binary_accuracy": binary_accuracy
    }




In [ ]:
## residual plots
import matplotlib.pyplot as plt

def plot_residuals(trues, preds, model_name='Model'):
    residuals = trues - preds
    plt.figure(figsize=(6, 4))
    plt.hist(residuals, bins=50, color='lightblue', edgecolor='black')
    plt.title(f"Residual Distribution for {model_name}")
    plt.xlabel("Prediction Error")
    plt.ylabel("Frequency")
    plt.axvline(0, color='red', linestyle='--')
    plt.show()
